## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-14-50-44 +0000,nyt,"Live Updates: Hurricane Melissa, Stronger Than...",https://www.nytimes.com/live/2025/10/28/weathe...
1,2025-10-28-14-50-00 +0000,nyt,Japanese Executives Called On by Trump Pledge ...,https://www.nytimes.com/2025/10/28/business/tr...
2,2025-10-28-14-49-17 +0000,nyt,U.S. Military Kills 14 More People Accused of ...,https://www.nytimes.com/2025/10/28/us/politics...
3,2025-10-28-14-48-30 +0000,nypost,Hurricane Melissa is so slow you can walk fast...,https://nypost.com/2025/10/28/world-news/hurri...
4,2025-10-28-14-41-23 +0000,nypost,Freed Israeli hostages take to the runway to o...,https://nypost.com/2025/10/28/world-news/freed...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,43
118,new,22
2,hurricane,16
3,melissa,15
8,jamaica,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
71,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...,101
156,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,95
78,2025-10-28-11-49-55 +0000,nypost,Toyota pledges $10B toward new auto plants in ...,https://nypost.com/2025/10/28/us-news/trump-to...,94
208,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,88
270,2025-10-27-19-37-06 +0000,nyt,"New College of Florida, a Conservative Bulwark...",https://www.nytimes.com/2025/10/27/us/politics...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
71,101,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...
232,75,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
127,59,2025-10-28-09-08-00 +0000,wsj,The Republican-led House Oversight Committee i...,https://www.wsj.com/politics/policy/republican...
317,51,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...
89,44,2025-10-28-11-10-16 +0000,nypost,Anti-Israel trans rabbi who was once tossed ou...,https://nypost.com/2025/10/28/us-news/anti-isr...
206,40,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
291,38,2025-10-27-18-01-17 +0000,nypost,Andrew Cuomo narrows gap against Zohran Mamdan...,https://nypost.com/2025/10/27/us-news/andrew-c...
318,38,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
178,35,2025-10-28-02-19-38 +0000,bbc,Brother of journalist killed in occupied West ...,https://www.bbc.com/news/articles/cx2dnz5zvgxo...
189,35,2025-10-28-01-00-00 +0000,wsj,Watering down dish soap. Trading down to groce...,https://www.wsj.com/economy/consumers/the-leng...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
